In [ ]:
from qiskit import * # Obavezno pokrenuti ovaj blok koda!
from qiskit.tools.visualization import plot_histogram # Potrebno je dodati ovu liniju koda kako bismo mogli prikazati histogram

# <center> 9.1 - Rješenja zadataka prije vježbe </center>
<hr>

### 6.4 - Zadatak 1 

Rješenje zadatka je slično primjerima. Ono izgleda ovako:

In [ ]:
qc = QuantumCircuit(1)

qc.x(0)
qc.h(0)
qc.x(0)
qc.z(0)

qc.draw("mpl")

### 7.4 - Zadatak 1 (Generator nasumičnih brojeva)

Zadatak nije toliko težak za implementirati. Dovoljno je postaviti 5 qubita u superpoziciju putem Hadamardove kapije, te ih sve izmjeriti.

Ideja iza ovoga zadatka jeste ta da bi kvantni računari mogli biti odlični nasumični generatori, jer svaki bit koji dolazi u obzir pri generisanju broja u perfektnom balansu između 0 i 1. Samim tim, nema pristrasnosti prema nekom broju.

Kako je u zadatku rečeno da izvršimo samo jedno mjerenje, jedini mogući izlaz iz rječnika ćemo uzeti i konvertovati u decimalni brojni sistem. <br>
Pogledajte kod ispod.

In [ ]:
generator = QuantumCircuit(5,5) # Kreiramo kvantno kolo koje ima 5 kvantnih bitova i 5 standardnih bitova

generator.h([0, 1, 2, 3, 4]) # Postavljamo Hadamardovu kapiju na svaki qubit
generator.barrier()          # Barijera je postavljeno čisto iz estetskih razloga
generator.measure([0, 1, 2, 3, 4], [0, 1, 2, 3, 4]) # Mjerimo qubite

In [ ]:
generator.draw("mpl") # Prikazujemo kvantno kolo

In [ ]:
backend = Aer.get_backend('qasm_simulator')            # Kreiramo simulator  
result = execute(generator, backend, shots=1).result() # Rezultate mjerenja čuvamo u varijabli "result" i pokrećemo samo jednom
counts = result.get_counts()                           

In [ ]:
plot_histogram(counts) # Prikažimo rezultat mjerenja

In [ ]:
binarni_broj = list(counts.keys())[0] # Prvo formiramo riječnik u listu, te uzimamo prvi broj iz nje - ujedno i jedini
decimalni_broj = int(binarni_broj, 2) # Pretvaramo binarni string u decimalni broj

print("Nasumično generisani broj je:", decimalni_broj)

## <center> 9.2 Rješenja zadataka iz vježbe </center>
<hr>

### Zadatak 1 

Zadatak od nas traži da generišemo 4 Bell state-a. Rješenje je relativno intuitivno:

In [ ]:
phi_1 = QuantumCircuit(2) # 1/sqrt(2) * (|00> + |11>)
phi_1.h(0)
phi_1.cx(0, 1)

phi_2 = QuantumCircuit(2) # 1/sqrt(2) * (|00> - |11>)
phi_2.h(0)
phi_2.cx(0,1)
phi_2.z(0)

psi_1 = QuantumCircuit(2) # 1/sqrt(2) * (|01> + |10>)
psi_1.h(0)
psi_1.cx(0, 1)
psi_1.x(1)

psi_2 = QuantumCircuit(2) # 1/sqrt(2) * (|01> - |10>)
psi_2.h(0)
psi_2.cx(0, 1)
psi_2.z(0)
psi_2.x(1)

In [ ]:
# Prikažimo kvantna kola!
display(phi_1.draw("mpl"))
display(phi_2.draw("mpl"))
display(psi_1.draw("mpl"))
display(psi_2.draw("mpl"))

### Zadatak 2

Da ponovimo, u zadatku je potrebno napraviti kvantno kolo od 3 qubita koji započinju u $|+\rangle$ stanju, za koje važi:
<ul>
    <li>Kada nakon mjerenja $q_0$ bude $|0\rangle$, $q_1$ mjeri $|0\rangle$, a $q_2$ mjeri $|1\rangle$</li>
    <li>Kada nakon mjerenja $q_0$ bude $|1\rangle$, $q_1$ mjeri $|1\rangle$, a $q_2$ mjeri $|0\rangle$</li>
</ul>

Iz ovoga vidimo da je, u oba slučaja, stanje prvog i drugog qubita isto, te da je stanje prvog i trećeg različito.

Ovde ćemo se poslužiti trikom vraćanja na osnovno stanje, koje će nam pomoći da postavimo qubite u takvo stanje da, pri korištenju CNOT-a, oni ili budu isti, ili različiti od stanja prvog qubita.

In [ ]:
qc_2 = QuantumCircuit(3, 3)
qc_2.h([0, 1, 2]) # Postavljamo qubite u početno stanje

qc_2.h([1, 2]) # Pri korištenju još jednog Hadamarda, vratićemo qubite u stanje |0>
qc_2.x(2)      # Obrnućemo treći qubit, tj. postavićemo ga u stanje |1>
qc_2.cx(0, 1)  # "Kopiramo" stanje sa prvog qubita na drugi
qc_2.cx(0, 2)  # Obrćemo stanje trećeg qubita u odnosu na prvi, jer treći i prvi qubit moraju imati različito stanje na kraju

In [ ]:
qc_2.draw("mpl") # Prikazujemo kvantno kolo

### Zadatak 3

`Kada kažemo prvi qubit, mislimo na prvi qubit sa desne strane, zbog "little endian" notacije`

Treći zadatak od nas zahtijeva da pokušamo rekonstruisati neku klasičnu funkciju, ali kao kvantno kolo. S obzirom da stanja $|0\rangle$ i $|1\rangle$ posmatramo kao klasične bitove $0$ i $1$, možemo razmišljati o njima na sličan način.

Pri rješavanju ovakvih zadataka jedino što je potrebno uraditi jeste prepoznati šablon ili obrazac po kojem se generiše izlaz. To, nekada, zna biti jako komplikovano, ali u ovome zadatku i nije. 

Pogledajmo mapiranje: 

$$000 \rightarrow 100$$
$$001 \rightarrow 011$$
$$010 \rightarrow 110$$
$$011 \rightarrow 001$$
$$100 \rightarrow 000$$
$$101 \rightarrow 111$$
$$110 \rightarrow 010$$
$$111 \rightarrow 101$$

Možemo napraviti sljedeće obzervacije:
</ol>
    <li> <b>Prvi qubit izlaza</b> ($q_0$) jednak je prvom qubitu ulaza ($q_0$)</li>
    <li> <b>Drugi qubit izlaza</b> ($q_1$) jednak je XOR-u između prvog i drugog qubita ulaza ($q_1$ izlaza = $q_0 \oplus q_1$ ulaza)</li>
    <li> <b>Treći qubit izlaza</b> ($q_2$) jednak je negaciji XOR-a između prvog i drugiog qubita ulaza ($q_2$ izlaza = $\neg(q_0 \oplus q_2)$)</li>
</ol>
Ne postoji neko detaljno objašenjenje u vezi prepoznavanja šablona - to je jedino moguće uz dosta vježbe i intuicije.

Pogledajte kod ispod.

In [ ]:
qc_3 = QuantumCircuit(3,3)

qc_3.cx(0,1) # Čuvamo (q_0 ^ q_1) u qubitu q_1
qc_3.cx(0,2) # Čuvamo (q_0 ^ q_2) u qubitu q_2
qc_3.x(2)    # Negiramo postojeći XOR

qc_4.barrier()

qc_3.measure([0, 1, 2], [0, 1, 2])
qc.draw("mpl")

### Zadatak 4

Slično kao zadatku 3, potrebno je da pronađemo šablon u sljedećem mapiranju:
    
    
$$000 \rightarrow 100$$
$$001 \rightarrow 001$$
$$010 \rightarrow 110$$
$$011 \rightarrow 011$$
$$100 \rightarrow 111$$
$$101 \rightarrow 010$$
$$110 \rightarrow 101$$
$$111 \rightarrow 000$$

Možemo da zaključimo:
<ol>
    <li> <b>Drugi qubit izlaza</b> ($q_1$) jednak je XOR-u između trećeg i drugog qubita ulaza ($q_1$ izlaza = $q_2 \oplus q_1$ ulaza)</li>
    <li> <b>Prvi qubit izlaza</b> ($q_0$) jednak je XOR-u između trećeg i prvog qubita ulaza ($q_0$ izlaza = $q_2 \oplus q_0$ ulaza)</li>
    <li> <b>Treći qubit izlaza</b> ($q_2$) jednak je negaciji XOR-a između prvog qubita <b>izlaza</b> i trećeg qubita ulaza ($q_2$ (izlaza) = $q_0 (izlaza) \oplus q_2 (ulaza)$)</li>
</ol>

In [ ]:
qc_4 = QuantumCircuit(3, 3)
qc_4.h([0, 1, 2])

qc_4.cx(2,1) # Čuvamo (q_2 ^ q_1) u qubitu q_1
qc_4.cx(2,0) # Čuvamo (q_2 ^ q_0) u qubitu q_0
qc_4.cx(0,2) # Čuvamo (q_0 (izlaza) ^ q_2 (ulaza))
qc_4.x(2)    # Negiramo q_2, pri čemu on prelazi u finalno stanje izlaza

qc_4.barrier()

qc_4.measure([0, 1, 2], [0, 1, 2])
qc_4.draw("mpl")

### Zadatak 5 

Kako je u tekstu zadatka objašnjen redoslijed operacija, dovoljno je da ih samo implementiramo:

In [ ]:
qc_5 = QuantumCircuit(2,1)

# Prvi korak
qc_5.x(1)      # Postavljamo q_1 u stanje |1>, kako bi on nakon Hadamardove kapije postao |->
qc_5.h([0, 1]) # Apliciramo Hadamardove kapije te kreiramo stanje sistema: |+->

# Drugi korak - Nadovezujemo na naše kolo orakl iz teksta zadatka
qc_5.barrier()
qc_5.cx(0,1)
qc_5.x(1)
qc_5.cx(0,1)
qc_5.barrier()

# Treći korak - Vraćamo stanje prvog qubita Hadamardovom kapijom
qc_5.h(0)

# Četvrti korak - Mjerimo prvi qubit da bismo saznali da li je funkcija konstantna ili balansirana
qc_5.measure(0, 0)

In [ ]:
qc_5.draw("mpl") # Prikazujemo kvantno kolo

In [ ]:
backend = Aer.get_backend('qasm_simulator')          # Kreiramo simulator  
result = execute(qc_5, backend, shots=1024).result() # Rezultate mjerenja čuvamo u varijabli "result" i pokrećemo samo jednom
counts = result.get_counts()                           

In [ ]:
plot_histogram(counts)

<center> <h4> Zaključujemo da je funckija konstantna! </h4> </center>

<b>Preporuka:</b> Iako Dojčov algoritam možda ne izgleda toliko zanimljivo, postoje još dva relativno jednostavna kvantna algoritma koji ne zahtijevaju nešto matematike, a to su:
<ul>
    <li> <b>Bernstein-Vazirani algoritam</b> - koji može da pogodi <b>bilo koji broj iz prvog pokušaja</b>, za razliku od klasičnih računara kojima bi trebalo prosječno makar $N$ pokušaja. Pogledajte tutorijal <a href="https://www.youtube.com/watch?v=sqJIpHYl7oo"> ovdje </a></li>
    <li> <b>Groverov algoritam</b> - koji može da pronađe otpimalno rješenje u logaritamskom vremenu, što je dosta brže u odnosu na klasične računare, kojima treba prosječno $\frac{N}{2}$ pokušaja. Na primjer, možete ga koristiti da pronađete optimalno rješenje za Sudoku: <a href="https://qiskit.org/textbook/ch-algorithms/grover.html#5.-Solving-Sudoku-using-Grover's-Algorithm-"> Kliknte ovdje </a></li>
<ul>